In [1]:
import pandas as pd

### 导入数据

In [2]:
f = open('C:/Users/cyb/ipnb/haha/280_Ben_八大直播八大案例配套课件/第一章/RFM_TRAD_FLOW(1).csv')
trad_flow = pd.read_csv(f)

In [3]:
trad_flow.head(10)

,transID,cumid,time,amount,type_label,type
0,9407,10001,14JUN09:17:58:34,199.0,正常,Normal
1,9625,10001,16JUN09:15:09:13,369.0,正常,Normal
2,11837,10001,01JUL09:14:50:36,369.0,正常,Normal
3,26629,10001,14DEC09:18:05:32,359.0,正常,Normal
4,30850,10001,12APR10:13:02:20,399.0,正常,Normal
5,32007,10001,04MAY10:16:45:58,269.0,正常,Normal
6,36637,10001,04JUN10:20:03:06,0.0,赠送,Presented
7,43108,10001,06JUL10:16:56:40,381.0,正常,Normal
8,43877,10001,10JUL10:20:41:54,-399.0,退货,returned_goods
9,46081,10001,23JUL10:16:35:45,0.0,赠送,Presented


#### transID: 商品 id ， cumid： 用户 id， type 有四种，Normal, Presented, Special_offer, returned_goods 分别是 正常，赠送，特价，退货

### 通过 RFM 方法，建立模型
1. 通过计算 F 反映客户对打折产品的偏好

In [4]:
F = trad_flow.groupby(['cumid', 'type'])[['transID']].count()
F.head()

transID
cumid type                   
10001 Normal               15
      Presented             8
      Special_offer         2
      returned_goods        2
10002 Normal               12

In [5]:
# 对 F 表进行转置，复合一般阅读习惯
F_trans = pd.pivot_table(index='cumid', columns='type', data=F, values='transID')
F_trans.head()

type,Normal,Presented,Special_offer,returned_goods
cumid,,,,
10001,15.0,8.0,2.0,2.0
10002,12.0,5.0,NaN,1.0
10003,15.0,8.0,1.0,1.0
10004,15.0,12.0,2.0,1.0
10005,8.0,5.0,NaN,1.0


In [6]:
# 表中 Special_offer 出现 nan ，进行填充
F_trans['Special_offer'] = F_trans['Special_offer'].fillna(0)
F_trans.head()

type,Normal,Presented,Special_offer,returned_goods
cumid,,,,
10001,15.0,8.0,2.0,2.0
10002,12.0,5.0,0.0,1.0
10003,15.0,8.0,1.0,1.0
10004,15.0,12.0,2.0,1.0
10005,8.0,5.0,0.0,1.0


In [7]:
# 对打折产品的兴趣高低 这里的计算方法是用 打折商品除以打折和正常价格商品的总和
F_trans['interest'] = F_trans['Special_offer'] / (F_trans['Special_offer'] + F_trans['Normal'])
F_trans.head()

type,Normal,Presented,Special_offer,returned_goods,interest
cumid,,,,,
10001,15.0,8.0,2.0,2.0,0.117647
10002,12.0,5.0,0.0,1.0,0.000000
10003,15.0,8.0,1.0,1.0,0.062500
10004,15.0,12.0,2.0,1.0,0.117647
10005,8.0,5.0,0.0,1.0,0.000000


2. 通过计算 M 反应客户的价值信息

In [8]:
M = trad_flow.groupby(['cumid', 'type'])[['amount']].sum()
# 对 F 表进行转置，复合一般阅读习惯
M_trans = pd.pivot_table(index='cumid', columns='type', data=M, values='amount')
# 表中 Special_offer, returned_goods 出现 nan ，进行填充
M_trans['Special_offer'] = M_trans['Special_offer'].fillna(0)
M_trans['returned_goods'] = M_trans['returned_goods'].fillna(0)
# 总收入是正常价格商品+打折商品+退货商品价格（这里是负的）
M_trans["value"]=M_trans['Normal']+M_trans['Special_offer']+M_trans['returned_goods']
M_trans.head()

type,Normal,Presented,Special_offer,returned_goods,value
cumid,,,,,
10001,3608.0,0.0,420.0,-694.0,3334.0
10002,1894.0,0.0,0.0,-242.0,1652.0
10003,3503.0,0.0,156.0,-224.0,3435.0
10004,2979.0,0.0,373.0,-40.0,3312.0
10005,2368.0,0.0,0.0,-249.0,2119.0


3. 通过计算R反应客户是否为沉默客户

In [9]:
# 首先修改时间格式
from datetime import datetime
import time

def to_time(t):
    out_t = time.mktime(time.strptime(t, '%d%b%y:%H:%M:%S'))### 此处改为时间戳是为了方便后面的 qcut 函数分箱
    return out_t

In [10]:
a="14JUN09:17:58:34"
print(to_time(a))

1244973514.0


In [11]:
trad_flow['time_new'] = trad_flow['time'].apply(to_time)
trad_flow.head()

,transID,cumid,time,amount,type_label,type,time_new
0,9407,10001,14JUN09:17:58:34,199.0,正常,Normal,1.244974e+09
1,9625,10001,16JUN09:15:09:13,369.0,正常,Normal,1.245136e+09
2,11837,10001,01JUL09:14:50:36,369.0,正常,Normal,1.246431e+09
3,26629,10001,14DEC09:18:05:32,359.0,正常,Normal,1.260785e+09
4,30850,10001,12APR10:13:02:20,399.0,正常,Normal,1.271049e+09


In [12]:
R = trad_flow.groupby(['cumid'])[['time_new']].max()
R.head()

,time_new
cumid,
10001,1.284699e+09
10002,1.278129e+09
10003,1.282983e+09
10004,1.283057e+09
10005,1.282127e+09


### 构建模型，筛选目标客户
 - 通过构建客户对打折产品的兴趣高低，客户价值，客户最后消费时间三个指标的二分类，来进行评估

In [13]:
from sklearn import preprocessing

In [20]:
# 分为两组，两份样本数相同，最后的[1]实际上取第一个数
threshold = pd.qcut(F_trans['interest'], 2, retbins=True)[1][1]
binarizer = preprocessing.Binarizer(threshold=threshold)
# 这里 Binarizer 只能处理 2D 数据，所有使用 reshape 转换
interest_q = pd.DataFrame(binarizer.transform(F_trans['interest'].values.reshape(-1, 1)))
interest_q.index = F_trans.index
interest_q.columns = ['interest']

In [21]:
# 对 M 的数据进行二分类
threshold = pd.qcut(M_trans['value'], 2, retbins=True)[1][1]
binarizer = preprocessing.Binarizer(threshold=threshold)
value_q = pd.DataFrame(binarizer.transform(M_trans['value'].values.reshape(-1, 1)))
value_q.index = M_trans.index
value_q.columns = ['value']
# 对 R 的数据进行二分类
threshold = pd.qcut(R["time_new"], 2, retbins=True)[1][1]
binarizer = preprocessing.Binarizer(threshold=threshold)
time_new_q = pd.DataFrame(binarizer.transform(R["time_new"].values.reshape(-1, 1)))
time_new_q.index=R.index
time_new_q.columns=["time"]

In [22]:
# 按照列进行合并，用 concat 方法
analysis = pd.concat([interest_q, value_q, time_new_q], axis=1)

In [23]:
analysis.head()

,interest,value,time
cumid,,,
10001,1.0,1.0,1.0
10002,0.0,0.0,0.0
10003,0.0,1.0,0.0
10004,1.0,1.0,0.0
10005,0.0,0.0,0.0


### 将模型转化为便于理解的形式

In [25]:
label = {
    (0,0,0):'无兴趣-低价值-沉默',
    (1,0,0):'有兴趣-低价值-沉默',
    (1,0,1):'有兴趣-低价值-活跃',
    (0,0,1):'无兴趣-低价值-活跃',
    (0,1,0):'无兴趣-高价值-沉默',
    (1,1,0):'有兴趣-高价值-沉默',
    (1,1,1):'有兴趣-高价值-活跃',
    (0,1,1):'无兴趣-高价值-活跃'
}
analysis['label'] = analysis[['interest','value','time']].apply(lambda x: label[(x[0],x[1],x[2])], axis = 1)
analysis.head()

,interest,value,time,label
cumid,,,,
10001,1.0,1.0,1.0,有兴趣-高价值-活跃
10002,0.0,0.0,0.0,无兴趣-低价值-沉默
10003,0.0,1.0,0.0,无兴趣-高价值-沉默
10004,1.0,1.0,0.0,有兴趣-高价值-沉默
10005,0.0,0.0,0.0,无兴趣-低价值-沉默
